In [1]:
# file by aya abdalsalam

import pandas as pd
import string
import re
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textblob

from textblob import TextBlob


#Downloading packages 
nltk.download('stopwords')

pd.set_option('display.max_colwidth', None)

reddit = pd.read_csv('reddit.csv')
gab = pd.read_csv('gab.csv')
print(gab.shape)
print(reddit.shape)


df = pd.concat([reddit,gab])

df.isna().sum()
df = df.dropna()
df = df.drop(['response','id','hate_speech_idx'],axis =1)


def clean_text(text):
    '''
    remove digits, punctuation, special characters, extra spaces, links
    '''

    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[ð]', '', text)
    text = re.sub('\w*[\d@]\w*', '', text)
    text = re.sub('[A-Za-z]+\d+|\d+[A-Za-z]+', '', text).strip()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('^RT[\s]+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\±\ð\$\%\&\@+\"\'\,', '', text)
    return text


# Applying the cleaning function to both test and training datasets
df['text'] = df['text'].apply(
    lambda x: clean_text(x))

#df['response'] = df['response'].apply(lambda x: clean_text(x))

# Let's take a look at the updated text
df.sample()

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


for column in df:
    for i in column:
        remove_emoji(i)
labels = []

for i in df['text']:
    score = textblob.TextBlob(i).polarity
    if score >= 0:
        label = 0
    elif score < 0 and score >= -0.5:
        label = 1
    elif score < -0.5:
        label = 2
    labels.append(label)
df['label'] = labels

def removeStopWords(df):
    stop_words = stopwords.words('english')

    df["text"] = df["text"].apply(lambda x: " ".join(
        [word for word in word_tokenize(x) if word.lower() not in stop_words]))

    # lemmatization
    lemmatizer = WordNetLemmatizer()
    df["text"] = df["text"].apply(
        lambda x: " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

    return df

df = removeStopWords(df)
df


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(11825, 4)
(5020, 4)


,text,label
0,subsection retarded Hungarians Ohh boy brace livid Bulbasaur coming trying hate hole stupider countryman Hiii got work mainly typa guy imagine writing US stupid site basically try fit Western European US big city hipster Rich kid Budapest feel brave journalistic hero wow guess soyboys every country Owen Benjamins soyboy song go every country thats amazing,0
1,yall hear sumn mean live small town rn work people originally mining community one protesting Nobody mentioned protest today Nobody care theyre busy loving family working hard Thank god electoral college Liberals think dont matter ah liberal slipped tell first question asked loaded retarded working hard mean political view get validated much manual labour put say yes course clue working hard mean soyboys Based comment regarding EC guess dont really think mil majority people voted Hillary dont really matter shouldnt explain point EC want talk someone patient check AskTheDonald,0
3,Japanese arent retarded know fictional character arent owed anything much le dignity Japan doesnt shy away lot thing Male character smack around woman humor something like Seven Deadly Sins main character frequently groping woman stealing panty etc,1
4,might true didnt example whole segment US population blindly screaming head man based accusation demonstrably unreliable witness event year ago nobody could even begin corroborate even witness scary time men fuckwits like Author Honestly believe Kavanaugh probably say Especially information heavy drinking bar fight came Heres thing evidence presented ie none testimony last week would nowhere near enough convict anything even still prosecutable fuck anyone think bar job supreme court please give presumption innocence anyone get he actually proven guilty may seem unfair retarded reason Id personally love presumption innocence still thing day Im accused something didnt Im saying didnt Im saying presume people innocent god damn fucking reason fact lied multiple time front Senate read speech could seriously written grader pretty fucking repulsive reason alone GOP ditched lie avoid embarassment dont want somebody like highest court country already embarassed point anyways halfretarded speech fuck anyone think bar job supreme court please give presumption innocence anyone get he actually proven guilty According detractor job interview innocent proven guilty doesnt apply job interview dont care accusation criminal decade ago require testimony oath Needless say dont explain standard using unless count possible shouldnt SCOTUS folk already embarassed point anyways halfretarded speech mean speech made upset obvious reason including death threat family remotely job SCOTUS like isnt guilt innocence establishing character Oh commiting perjury repeatedly Establishing character based unverifiable accusation crime would older Fox Broadcasting Network FBI investigating wasnt guilt innocence course vague perjury cry moving goalpost kind ironic detractor would care much truth backing story nobody prove Perjury perjury establishes character devil triangle Hes judge willing lie oath petty bullshit Moving goalpost job interview goal always establish apropriate job answer many reason least naked partisanship devil triangle Hes judge willing lie oath petty bullshit couldnt get thing decade ago wrong petty want head Moving goalpost job interview often job interview involve people testifying oath government official crime someone allegedly committed decade ago hearing Job interview people said want excuse lower standard evidence goal always establish apropriate job goal try get rejected unprovable accusation maybe stall long enough Dems win enough seat midterm block conventionally Maybe even hurt GOPs midterm chance keep bleating establishing character keep ignoring Fords actual accusation youre moving goalpost Youre acting like way man act he accused rape yelled hundred thousand people gotten death threat family normally behaves answer many reason least naked 

In [2]:
df.duplicated().sum()
df = df.drop_duplicates()
df.duplicated().sum()


0

In [3]:
df.to_csv('Hate_speech_dataset_gabAndReddit.csv')

In [4]:
#df2 = pd.read_csv('Hate_speech_dataset_gabAndReddit.csv')
#df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
#df2.to_csv('A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech.csv')
